Load data in Tensorflow.

In [1]:
import codecs
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

/Users/pasquale/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/pasquale/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
training_data_folder = '../training_data/web-radio/output/rec'
embDir = '../embeddings'
what = 'artist'

uri_file = '%s/%s.emb.u' % (embDir, what)
vector_file = '%s/%s.emb.v' % (embDir, what)
# header_file = '%s/%s.emb.h' % (embDir, what)
training_file = '%s/%s.dat' % (training_data_folder, what)

vectors = np.array([line.strip().split(' ') for line in codecs.open(vector_file, 'r', 'utf-8')])
# heads = np.array([line.strip() for line in codecs.open(header_file, 'r', 'utf-8')])
uris = np.array([line.strip() for line in codecs.open(uri_file, 'r', 'utf-8')])

train_array = np.array([line.strip().split(' ') for line in codecs.open(training_file, 'r', 'utf-8')])
train_array.shape

(11815, 3)

In [3]:
def get_embs(x):
    v = vectors[np.argwhere(uris == x)]
    if v.size == 0:
        result = -2. * np.ones(vectors[0].size)
    else:
        result = v[0][0]
    return result.astype('float32')

In [4]:
col1 = np.array([get_embs(xi) for xi in train_array[:, 0]])
col2 = np.array([get_embs(xi) for xi in train_array[:, 1]])
col3 = np.array(train_array[:, 2]).astype('float32')
col3 = col3.reshape((col3.size, 1))

training_vector = np.concatenate((col1, col2, col3), axis=1)

train, test = train_test_split(training_vector, train_size=0.3)

train_vector = train[:, :-1]
train_label = train[:, -1]

test_vector = test[:, :-1]
test_label = test[:, -1]

/Users/pasquale/anaconda3/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [5]:
print(col1.shape)
print(col2.shape)
print(col3.shape)

print(training_vector.shape)
print(test.shape)

print(train_vector.shape)
print(train_label.shape)

(11815, 14)
(11815, 14)
(11815, 1)
(11815, 29)
(8271, 29)
(3544, 28)
(3544,)


In [36]:
# Parameters
learning_rate = 0.1
num_steps = 1000
batch_size = 1
display_step = 1

# Network Parameters
n_hidden_1 = 256  # 1st layer number of neurons
n_hidden_2 = 256  # 2nd layer number of neurons
num_input = train_vector[0].size
num_output = int(num_input / 2)

# tf Graph input
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_output])

In [37]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_output]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_output]))
}

In [38]:
# Create model
def neural_net(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [39]:
def similarity_loss(_sentinel=None, labels=None, logits=None, dim=-1, name=None):
    l = col1[0].size
    a = train_vector[:, 0:l]
    b = train_vector[:, l:]

    max_distance = weightedL2(np.ones(l), tf.constant(np.ones(l) * -1), logits)

    predicted = compute_sim(a, b, logits, max_distance)
    expected = tf.constant(train_label)

    loss = tf.subtract(expected, predicted)
    loss = tf.reduce_mean(loss)
    return loss


def weightedL2(a, b, w=1):
    # https://stackoverflow.com/a/8861999/1218213
    q = tf.subtract(a, b)
    # return np.sqrt((w * q * q).sum())
    pow_q = tf.cast(tf.pow(q, 2), tf.float32)
    _w = tf.reshape(w, [w.shape[1]])

    _sum = tf.reduce_sum(tf.multiply(_w, pow_q), axis=0, keepdims=True)
    return tf.reshape(_sum, [1, 1])


def compute_sim(seed, target, w, max_distance):
    lt = seed.shape[0]
    cost = [compute_similarity(seed[i], target[i], w, max_distance) for i in range(0, lt)]
    return tf.convert_to_tensor(cost, tf.float32)


def compute_similarity(seed, target, w, max_distance):
    b1 = np.argwhere(seed >= -1)
    b1.reshape(b1.size)
    b2 = np.argwhere(target >= -1)
    b2.reshape(b2.size)

    good_pos = np.intersect1d(b1, b2)
    if len(good_pos) == 0:
        return tf.constant(0.0, shape=[1, 1])

    _seed = seed[good_pos]
    _target = target[good_pos]
    _w = tf.gather(w, good_pos, axis=1)

    # distance
    d = weightedL2(_seed, _target, _w)

    # how much info I am not finding
    penalty = (len(b1) - len(good_pos)) / len(seed)
    multiplier = 1. - penalty

    # score
    s = tf.divide(tf.subtract(max_distance, d), max_distance)
    return tf.multiply(s, multiplier)


In [40]:
# Construct model
logits = neural_net(X)

# Define loss and optimizer
loss_op = similarity_loss(logits=logits, labels=Y)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

logits
(?, 14)
/
(?, 14)
(?, 11)
(?, 7)
(?, 8)
(?, 5)
(?, 4)
(?, 8)
(?, 3)
(?, 4)
(?, 8)
(?, 5)
(?, 11)
(?, 11)
(?, 11)
(?, 7)
(?, 11)
(?, 11)
(?, 4)
(?, 8)
(?, 11)
(?, 11)
(?, 11)
(?, 11)
(?, 5)
(?, 5)
(?, 11)
(?, 8)
(?, 5)
(?, 5)
(?, 4)
(?, 11)
(?, 3)
(?, 6)
(?, 5)
(?, 8)
(?, 8)
(?, 11)
(?, 11)
(?, 11)
(?, 5)
(?, 11)
(?, 11)
(?, 7)
(?, 8)
(?, 5)
(?, 11)
(?, 7)
(?, 3)
(?, 7)
(?, 11)
(?, 3)
(?, 14)
(?, 5)
(?, 8)
(?, 8)
(?, 11)
(?, 3)
(?, 11)
(?, 8)
(?, 8)
(?, 5)
(?, 11)
(?, 5)
(?, 11)
(?, 11)
(?, 5)
(?, 11)
(?, 3)
(?, 3)
(?, 11)
(?, 4)
(?, 8)
(?, 8)
(?, 8)
(?, 11)
(?, 11)
(?, 11)
(?, 5)
(?, 11)
(?, 11)
(?, 5)
(?, 4)
(?, 5)
(?, 11)
(?, 5)
(?, 5)
(?, 5)
(?, 14)
(?, 11)
(?, 11)
(?, 7)
(?, 7)
(?, 8)
(?, 5)
(?, 11)
(?, 11)
(?, 4)
(?, 11)
(?, 11)
(?, 5)
(?, 11)
(?, 11)
(?, 11)
(?, 7)
(?, 8)
(?, 11)
(?, 11)
(?, 8)
(?, 11)
(?, 5)
(?, 3)
(?, 5)
(?, 8)
(?, 11)
(?, 5)
(?, 11)
(?, 3)
(?, 8)
(?, 7)
(?, 5)
(?, 14)
(?, 11)
(?, 11)
(?, 11)
(?, 7)
(?, 11)
(?, 11)
(?, 8)
(?, 11)
(?, 7)
(?, 7)
(?, 8)
(?,

(?, 4)
(?, 11)
(?, 5)
(?, 8)
(?, 11)
(?, 7)
(?, 5)
(?, 8)
(?, 11)
(?, 4)
(?, 11)
(?, 11)
(?, 11)
(?, 5)
(?, 11)
(?, 3)
(?, 11)
(?, 4)
(?, 11)
(?, 8)
(?, 8)
(?, 11)
(?, 11)
(?, 3)
(?, 11)
(?, 11)
(?, 11)
(?, 8)
(?, 11)
(?, 4)
(?, 5)
(?, 7)
(?, 11)
(?, 11)
(?, 11)
(?, 11)
(?, 11)
(?, 3)
(?, 8)
(?, 11)
(?, 7)
(?, 4)
(?, 5)
(?, 7)
(?, 11)
(?, 5)
(?, 11)
(?, 7)
(?, 11)
(?, 11)
(?, 5)
(?, 8)
(?, 5)
(?, 3)
(?, 8)
(?, 11)
(?, 5)
(?, 5)
(?, 11)
(?, 11)
(?, 8)
(?, 11)
(?, 11)
(?, 8)
(?, 8)
(?, 8)
(?, 8)
(?, 11)
(?, 8)
(?, 8)
(?, 4)
(?, 7)
(?, 8)
(?, 7)
(?, 11)
(?, 5)
(?, 5)
(?, 8)
(?, 8)
(?, 8)
(?, 5)
(?, 7)
(?, 11)
(?, 11)
(?, 3)
(?, 11)
(?, 8)
(?, 11)
(?, 4)
(?, 14)
(?, 7)
(?, 3)
(?, 7)
(?, 4)
(?, 8)
(?, 11)
(?, 11)
(?, 11)
(?, 5)
(?, 3)
(?, 5)
(?, 3)
(?, 11)
(?, 8)
(?, 8)
(?, 11)
(?, 8)
(?, 7)
(?, 11)
(?, 11)
(?, 11)
(?, 11)
(?, 7)
(?, 3)
(?, 11)
(?, 5)
(?, 8)
(?, 8)
(?, 11)
(?, 8)
(?, 8)
(?, 11)
(?, 11)
(?, 11)
(?, 4)
(?, 7)
(?, 4)
(?, 11)
(?, 7)
(?, 11)
(?, 7)
(?, 5)
(?, 11)
(?, 11)
(?, 7)


(?, 5)
(?, 8)
(?, 7)
(?, 4)
(?, 14)
(?, 8)
(?, 11)
(?, 11)
(?, 7)
(?, 5)
(?, 11)
(?, 8)
(?, 4)
(?, 5)
(?, 11)
(?, 11)
(?, 11)
(?, 11)
(?, 11)
(?, 8)
(?, 5)
(?, 7)
(?, 11)
(?, 11)
(?, 11)
(?, 5)
(?, 5)
(?, 4)
(?, 11)
(?, 11)
(?, 3)
(?, 11)
(?, 4)
(?, 8)
(?, 5)
(?, 11)
(?, 5)
(?, 5)
(?, 3)
(?, 5)
(?, 5)
(?, 7)
(?, 8)
(?, 7)
(?, 14)
(?, 3)
(?, 7)
(?, 5)
(?, 8)
(?, 7)
(?, 3)
(?, 5)
(?, 14)
(?, 7)
(?, 11)
(?, 11)
(?, 11)
(?, 14)
(?, 11)
(?, 11)
(?, 11)
(?, 5)
(?, 7)
(?, 11)
(?, 8)
(?, 3)
(?, 4)
(?, 11)
(?, 7)
(?, 11)
(?, 8)
(?, 5)
(?, 11)
(?, 3)
(?, 6)
(?, 11)
(?, 3)
(?, 8)
(?, 8)
(?, 8)
(?, 5)
(?, 3)
(?, 11)
(?, 11)
(?, 11)
(?, 11)
(?, 7)
(?, 5)
(?, 7)
(?, 7)
(?, 8)
(?, 7)
(?, 11)
(?, 3)
(?, 11)
(?, 11)
(?, 8)
(?, 5)
(?, 5)
(?, 7)
(?, 8)
(?, 3)
(?, 11)
(?, 3)
(?, 11)
(?, 5)
(?, 4)
(?, 5)
(?, 11)
(?, 11)
(?, 11)
(?, 11)
(?, 7)
(?, 8)
(?, 7)
(?, 7)
(?, 5)
(?, 4)
(?, 7)
(?, 11)
(?, 11)
(?, 5)
(?, 4)
(?, 7)
(?, 14)
(?, 7)
(?, 5)
(?, 11)
(?, 8)
(?, 7)
(?, 11)
(?, 11)
(?, 5)
(?, 7)
(?, 11)
(?, 1

(?, 11)
(?, 3)
(?, 3)
(?, 5)
(?, 11)
(?, 8)
(?, 11)
(?, 7)
(?, 11)
(?, 8)
(?, 7)
(?, 3)
(?, 4)
(?, 5)
(?, 5)
(?, 5)
(?, 5)
(?, 11)
(?, 5)
(?, 8)
(?, 3)
(?, 3)
(?, 3)
(?, 4)
(?, 5)
(?, 3)
(?, 8)
(?, 8)
(?, 7)
(?, 4)
(?, 11)
(?, 3)
(?, 11)
(?, 8)
(?, 11)
(?, 7)
(?, 11)
(?, 4)
(?, 5)
(?, 8)
(?, 5)
(?, 3)
(?, 7)
(?, 11)
(?, 4)
(?, 5)
(?, 11)
(?, 8)
(?, 8)
(?, 8)
(?, 11)
(?, 11)
(?, 11)
(?, 5)
(?, 11)
(?, 4)
(?, 11)
(?, 4)
(?, 11)
(?, 11)
(?, 11)
(?, 7)
(?, 4)
(?, 7)
(?, 11)
(?, 11)
(?, 11)
(?, 8)
(?, 11)
(?, 4)
(?, 11)
(?, 11)
(?, 5)
(?, 11)
(?, 14)
(?, 11)
(?, 7)
(?, 11)
(?, 7)
(?, 11)
(?, 11)
(?, 8)
(?, 8)
(?, 11)
(?, 8)
(?, 11)


In [41]:
def next_batch(num, data, labels):

    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = data[idx]
    labels_shuffle = labels[idx]
    return data_shuffle, labels_shuffle

In [42]:
fake_labels = np.ones((train_vector.shape[0], num_output), dtype='float32')
fake_labels.shape

(3544, 14)

In [43]:
with tf.Session() as sess:
    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps + 1):
        batch_x, batch_y = next_batch(batch_size, train_vector, fake_labels)

        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for MNIST test images
    print("Testing Accuracy:",
          sess.run(accuracy, feed_dict={X: test_vector, Y: test_label}))

(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 1, Minibatch Loss= 0.0326, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 2, Minibatch Loss= 0.0310, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 3, Minibatch Loss= 0.0305, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 4, Minibatch Loss= 0.0288, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 5, Minibatch Loss= 0.0301, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 6, Minibatch Loss= 0.0290, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 7, Minibatch Loss= 0.0297, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 8, Minibatch Loss= 0

Step 64, Minibatch Loss= 0.0228, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 65, Minibatch Loss= -0.2216, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 66, Minibatch Loss= 0.0342, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 67, Minibatch Loss= 0.0329, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 68, Minibatch Loss= 0.0321, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 69, Minibatch Loss= 0.0317, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 70, Minibatch Loss= 0.0314, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 71, Minibatch Loss= 0.0312, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.f

Step 127, Minibatch Loss= 0.0302, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 128, Minibatch Loss= 0.0302, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 129, Minibatch Loss= 0.0302, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 130, Minibatch Loss= 0.0302, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 131, Minibatch Loss= 0.0301, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 132, Minibatch Loss= 0.0302, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 133, Minibatch Loss= 0.0301, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 134, Minibatch Loss= 0.0301, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class '

Step 190, Minibatch Loss= 0.0295, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 191, Minibatch Loss= 0.0295, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 192, Minibatch Loss= 0.0294, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 193, Minibatch Loss= 0.0294, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 194, Minibatch Loss= 0.0294, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 195, Minibatch Loss= 0.0294, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 196, Minibatch Loss= 0.0292, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 197, Minibatch Loss= 0.0293, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class '

Step 253, Minibatch Loss= 0.0306, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 254, Minibatch Loss= 0.0304, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 255, Minibatch Loss= 0.0304, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 256, Minibatch Loss= 0.0303, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 257, Minibatch Loss= 0.0303, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 258, Minibatch Loss= 0.0303, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 259, Minibatch Loss= 0.0304, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 260, Minibatch Loss= 0.0303, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class '

...

Step 757, Minibatch Loss= 0.0308, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 758, Minibatch Loss= 0.0308, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 759, Minibatch Loss= 0.0308, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 760, Minibatch Loss= 0.0308, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 761, Minibatch Loss= 0.0308, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 762, Minibatch Loss= 0.0308, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 763, Minibatch Loss= 0.0308, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 764, Minibatch Loss= 0.0308, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class '

Step 820, Minibatch Loss= 0.0306, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 821, Minibatch Loss= 0.0306, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 822, Minibatch Loss= 0.0307, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 823, Minibatch Loss= 0.0307, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 824, Minibatch Loss= 0.0307, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 825, Minibatch Loss= 0.0307, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 826, Minibatch Loss= 0.0307, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 827, Minibatch Loss= 0.0307, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class '

Step 883, Minibatch Loss= 0.0305, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 884, Minibatch Loss= 0.0306, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 885, Minibatch Loss= 0.0306, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 886, Minibatch Loss= 0.0306, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 887, Minibatch Loss= 0.0306, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 888, Minibatch Loss= 0.0306, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 889, Minibatch Loss= 0.0306, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 890, Minibatch Loss= 0.0306, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class '

Step 946, Minibatch Loss= 0.0305, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 947, Minibatch Loss= 0.0305, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 948, Minibatch Loss= 0.0305, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 949, Minibatch Loss= 0.0305, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 950, Minibatch Loss= 0.0305, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 951, Minibatch Loss= 0.0305, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 952, Minibatch Loss= 0.0305, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class 'numpy.float32'>
(1, 14)
<class 'numpy.float32'>
Step 953, Minibatch Loss= 0.0304, Training Accuracy= 0.000
(1, 14)
(1, 28)
<class '

NameError: name 'test_labels' is not defined